In [ ]:
#! pip uninstall torch torchvision -y
#! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install -U transformers
! pip install -U simpletransformers  

In [2]:
import os
os.chdir("/content/drive/MyDrive/Contest/Amazon/Data/dataset")

In [3]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import pandas as pd
import transformers
import re
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
data_raw = pd.read_pickle("data_mini.pkl")

In [ ]:
!pip install contractions
import contractions

In [6]:
label = data_raw['BROWSE_NODE_ID'].values
data = data_raw.drop(columns=['BROWSE_NODE_ID']).values

In [7]:
def standardize_text(data):
  data = ' '.join(data)
  data = contractions.fix(data)
  data = re.sub(r"[^A-Za-z]", " ", data)
  data = data.lower()
  return data

In [8]:
data_processed = []
for d in data:
  data_processed.append(standardize_text(d))

In [ ]:
data_raw.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [9]:
from collections import Counter

In [10]:
temp = list(Counter(data_raw.BROWSE_NODE_ID.values).keys())

In [11]:
key = {}
for i in range(len(temp)):
  key[temp[i]] = i;

In [12]:
label_mod = [key[val] for val in label]

In [ ]:
print(f"BROWSE_NODE_ID column has",  data_raw.BROWSE_NODE_ID.isnull().sum()/data_raw.shape[0]*100)
print(f"TITLE column has", data_raw.TITLE.isnull().sum()/data_raw.shape[0]*100)
print(f"DESCRIPTION column has", data_raw.DESCRIPTION.isnull().sum()/data_raw.shape[0]*100)
print(f"BULLET_POINTS column has", data_raw.BULLET_POINTS.isnull().sum()/data_raw.shape[0]*100)
print(f"BRAND column has", data_raw.BRAND.isnull().sum()/data_raw.shape[0]*100)

BROWSE_NODE_ID column has 0.0
TITLE column has 0.0
DESCRIPTION column has 0.0
BULLET_POINTS column has 0.0
BRAND column has 0.0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_processed, label_mod, test_size=0.2, random_state=42, stratify=label_mod)

In [14]:
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

train_df.shape, eval_df.shape

((4000, 2), (1000, 2))

In [16]:
from simpletransformers.classification import ClassificationModel
import logging
import sklearn

!export CUDA_LAUNCH_BLOCKING=1

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# They are lot of arguments to play with
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}
'''

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels = 50, args={'num_train_epochs':4, 'train_batch_size':32, 'max_seq_length':128}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)
print(result)


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

  0%|          | 0/4000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/125 [00:00<?, ?it/s]

...

Running Epoch 3 of 4:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

{'mcc': 0.8511515650912652, 'acc': 0.854, 'eval_loss': 0.5331617507934571}
